In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, colorchooser
import os
import re
import matplotlib.pyplot as plt
from datetime import datetime as dt
import pytz
import matplotlib.dates as mdates

# ★ ここから追加 ★
# お使いのPCにインストールされている日本語フォントを指定します
# (Windowsなら 'Yu Gothic' や 'Meiryo', 'MS Gothic' など)
# (Macなら 'Hiragino Sans' など)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Yu Gothic', 'Meiryo', 'Hiragino Sans', 'TakaoPGothic', 'IPAexGothic', 'Noto Sans CJK JP']
# グラフのマイナス記号が文字化けするのを防ぐ設定
plt.rcParams['axes.unicode_minus'] = False 
# ★ ここまで追加 ★

class GraphCreatorApp(tk.Tk):
    """
    グラフ一括作成ツールのメインアプリケーションクラス
    """
    def __init__(self):
        super().__init__()
        self.title("グラフ一括作成ツール")
        # ウィンドウサイズを縦に少し大きくする (タブが8個になるため)
        self.geometry("850x850") 

        style = ttk.Style(self)
        style.theme_use('clam')
        style.configure('TButton', padding=6, relief="flat", background="#ccc")
        style.configure('TLabel', padding=5)
        style.configure('TEntry', padding=5)
        style.configure('Header.TLabel', font=('Helvetica', 12, 'bold'))

        self.params = {}
        self.file_lists = {}
        self.graph_settings_vars = {}

        self.create_widgets()

    def create_widgets(self):
        """
        GUIウィジェットを作成し、配置します。
        """
        main_frame = ttk.Frame(self, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)

        # --- General Parameter Frame ---
        param_frame = ttk.LabelFrame(main_frame, text="全体パラメータ設定", padding="10")
        param_frame.grid(row=0, column=0, columnspan=2, sticky="ew", padx=5, pady=5)
        param_frame.columnconfigure(1, weight=1)


        ttk.Label(param_frame, text="ずらし時間 (msec):").grid(row=0, column=0, sticky="w")
        self.params['shift_time'] = tk.IntVar(value=30000)
        ttk.Entry(param_frame, textvariable=self.params['shift_time']).grid(row=0, column=1, sticky="ew")

        ttk.Label(param_frame, text="区切り時間 (msec):").grid(row=1, column=0, sticky="w")
        self.params['duration'] = tk.IntVar(value=30000)
        ttk.Entry(param_frame, textvariable=self.params['duration']).grid(row=1, column=1, sticky="ew")

        ttk.Label(param_frame, text="出力バージョン名:").grid(row=2, column=0, sticky="w")
        self.params['ver_number'] = tk.StringVar(value="HPF_peakcounts_1s")
        ttk.Entry(param_frame, textvariable=self.params['ver_number']).grid(row=2, column=1, sticky="ew")

        ttk.Label(param_frame, text="X軸 補助目盛り間隔(秒):").grid(row=3, column=0, sticky="w")
        self.params['x_minor_tick_interval'] = tk.IntVar(value=5)
        ttk.Entry(param_frame, textvariable=self.params['x_minor_tick_interval']).grid(row=3, column=1, sticky="ew")


        # --- Tabbed Settings Frame ---
        notebook = ttk.Notebook(main_frame)
        notebook.grid(row=1, column=0, columnspan=2, sticky="nsew", padx=5, pady=5)
        main_frame.grid_rowconfigure(1, weight=1)
        main_frame.grid_columnconfigure(1, weight=1)

        # グラフのデフォルト設定
        # ★ value2 を追加し、全グラフに use_secondary_y (第2軸使用フラグ) を追加
        graph_defaults = {
            'value':  {'name': '元波形',            'x': 1, 'y': 2, 'color': '#ff0000', 'ymin': -500,   'ymax': 500,    'auto': False, 'enabled': True,  'style': 'Line', 'use_secondary_y': False},
            'value2': {'name': 'カット後のifft',          'x': 1, 'y': 2, 'color': '#0080ff', 'ymin': -500,   'ymax': 500,    'auto': False, 'enabled': True, 'style': 'Line', 'use_secondary_y': False}, # ★ 新しいグラフ (デフォルトは主軸)
            'peak':   {'name': 'peak_value',          'x': 4, 'y': 5, 'color': '#0000ff', 'ymin': -500,   'ymax': 500,    'auto': True,  'enabled': False, 'style': 'Marker', 'use_secondary_y': True},
            'other':  {'name': 'peakcounts_1s',       'x': 1, 'y': 2, 'color': '#00ff00', 'ymin': 0,      'ymax': 100,    'auto': False, 'enabled': False,  'style': 'Line + Marker', 'use_secondary_y': True},
            'other2': {'name': 'revers_RRI',          'x': 1, 'y': 3, 'color': '#ffa500', 'ymin': 0,      'ymax': 0.5,    'auto': False, 'enabled': False, 'style': 'Line + Marker', 'use_secondary_y': True},
            'other3': {'name': 'shindenzu',           'x': 1, 'y': 2, 'color': '#800080', 'ymin': 0,      'ymax': 100,    'auto': True,  'enabled': False, 'style': 'Line', 'use_secondary_y': True},
            'other4': {'name': 'raw_variance_1.5s',   'x': 0, 'y': 1, 'color': '#008000', 'ymin': 40000,  'ymax': 200000, 'auto': False, 'enabled': False, 'style': 'Line + Marker', 'use_secondary_y': True},
            'other5': {'name': 'HPF_variance_0.5s',   'x': 0, 'y': 1, 'color': '#add8e6', 'ymin': 0,      'ymax': 3000,   'auto': False, 'enabled': False, 'style': 'Line + Marker', 'use_secondary_y': True}
        }
        
        self.file_labels = {}

        for i, (key, defaults) in enumerate(graph_defaults.items()):
            tab = ttk.Frame(notebook, padding="10")
            notebook.add(tab, text=f"グラフ {i+1} ({defaults['name']})")
            
            vars = {
                'enabled': tk.BooleanVar(value=defaults['enabled']),
                'name': tk.StringVar(value=defaults['name']),
                'x_axis': tk.IntVar(value=defaults['x']),
                'y_axis': tk.IntVar(value=defaults['y']),
                'autorange': tk.BooleanVar(value=defaults['auto']),
                'yrange_min': tk.DoubleVar(value=defaults['ymin']),
                'yrange_max': tk.DoubleVar(value=defaults['ymax']),
                'color': tk.StringVar(value=defaults['color']),
                'plot_style': tk.StringVar(value=defaults['style']),
                # ★ 第2軸 (以降) を使用するかの変数を追加
                'use_secondary_y': tk.BooleanVar(value=defaults.get('use_secondary_y', False))
            }

            # Enable/Disable Checkbox
            enabled_check = ttk.Checkbutton(tab, text="このグラフを有効にする", variable=vars['enabled'], command=lambda k=key: self.toggle_tab_state(k))
            enabled_check.grid(row=0, column=0, columnspan=3, sticky="w", pady=(0, 10))
            if key == 'value':
                enabled_check.config(state=tk.DISABLED) # グラフ1 (value) は常に有効

            # Settings entries
            ttk.Label(tab, text="グラフ名:").grid(row=1, column=0, sticky="w")
            ttk.Entry(tab, textvariable=vars['name']).grid(row=1, column=1, columnspan=2, sticky="ew")

            ttk.Label(tab, text="X軸の列:").grid(row=2, column=0, sticky="w")
            ttk.Entry(tab, textvariable=vars['x_axis'], width=5).grid(row=2, column=1, sticky="w")
            
            ttk.Label(tab, text="Y軸の列:").grid(row=3, column=0, sticky="w")
            ttk.Entry(tab, textvariable=vars['y_axis'], width=5).grid(row=3, column=1, sticky="w")

            autorange_check = ttk.Checkbutton(tab, text="Y軸オートレンジ", variable=vars['autorange'], command=lambda k=key: self.toggle_yrange_entries(k))
            autorange_check.grid(row=4, column=0, columnspan=2, sticky="w")

            ttk.Label(tab, text="Y軸 最小:").grid(row=5, column=0, sticky="w")
            vars['yrange_min_entry'] = ttk.Entry(tab, textvariable=vars['yrange_min'])
            vars['yrange_min_entry'].grid(row=5, column=1, sticky="w")

            ttk.Label(tab, text="Y軸 最大:").grid(row=6, column=0, sticky="w")
            vars['yrange_max_entry'] = ttk.Entry(tab, textvariable=vars['yrange_max'])
            vars['yrange_max_entry'].grid(row=6, column=1, sticky="w")
            
            ttk.Label(tab, text="線の色:").grid(row=7, column=0, sticky="w")
            color_frame = ttk.Frame(tab)
            color_frame.grid(row=7, column=1, columnspan=2, sticky="w")
            vars['color_label'] = ttk.Label(color_frame, text=defaults['color'], background=defaults['color'], width=10)
            vars['color_label'].pack(side=tk.LEFT, padx=5)
            ttk.Button(color_frame, text="色を選択", command=lambda k=key: self.select_color(k)).pack(side=tk.LEFT)

            ttk.Label(tab, text="プロット形式:").grid(row=8, column=0, sticky="w")
            style_combo = ttk.Combobox(tab, textvariable=vars['plot_style'], values=['Line', 'Marker', 'Line + Marker'], state='readonly', width=15)
            style_combo.grid(row=8, column=1, sticky="w")
            
            # ★ 'value' グラフ (主軸固定) 以外に「第2軸を使用する」チェックボックスを追加
            if key != 'value':
                vars['secondary_y_check'] = ttk.Checkbutton(tab, text="第2軸 (以降) を使用する", variable=vars['use_secondary_y'])
                vars['secondary_y_check'].grid(row=8, column=2, sticky="w", padx=(10, 0))


            ttk.Button(tab, text="ファイルを選択", command=lambda k=key: self.select_files(k)).grid(row=9, column=0, sticky="ew", pady=(10, 0))
            self.file_labels[key] = ttk.Label(tab, text="ファイルが選択されていません")
            self.file_labels[key].grid(row=9, column=1, columnspan=2, sticky="w", pady=(10, 0))

            self.graph_settings_vars[key] = vars
            self.toggle_yrange_entries(key)
            self.toggle_tab_state(key)

        # --- Run Button ---
        run_button = ttk.Button(main_frame, text="実行", command=self.run_processing)
        run_button.grid(row=2, column=0, columnspan=2, sticky="ew", padx=5, pady=10)
    
    def toggle_tab_state(self, key):
        vars = self.graph_settings_vars[key]
        state = tk.NORMAL if vars['enabled'].get() else tk.DISABLED
        
        for widget_key, widget in vars.items():
            if widget_key != 'enabled' and isinstance(widget, (ttk.Entry, ttk.Checkbutton, ttk.Button, ttk.Label, ttk.Combobox)):
                try:
                    # 'value' グラフには 'secondary_y_check' が存在しないため、キーの存在をチェック
                    if widget_key == 'secondary_y_check' and 'secondary_y_check' not in vars:
                        continue
                        
                    widget.config(state=state)
                except tk.TclError:
                    pass # Labels might not have a state option
                    
        if vars['enabled'].get():
            self.toggle_yrange_entries(key) # Re-apply yrange state
            vars['color_label'].config(background=vars['color'].get())
        else:
             vars['color_label'].config(background="#f0f0f0") # Default bg color when disabled


    def select_color(self, key):
        vars = self.graph_settings_vars[key]
        color_code = colorchooser.askcolor(title="色を選択")[1]
        if color_code:
            vars['color'].set(color_code)
            vars['color_label'].config(background=color_code, text=color_code)

    def toggle_yrange_entries(self, key):
        vars = self.graph_settings_vars[key]
        state = tk.DISABLED if vars['autorange'].get() else tk.NORMAL
        vars['yrange_min_entry'].config(state=state)
        vars['yrange_max_entry'].config(state=state)

    def select_files(self, file_type):
        files = filedialog.askopenfilenames(title=f"{file_type} ファイルを選択してください", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if files:
            self.file_lists[file_type] = sorted(list(files))
            self.file_labels[file_type].config(text=f"{len(files)}個のファイルを選択済み")
        else:
            self.file_lists.pop(file_type, None)
            self.file_labels[file_type].config(text="ファイルが選択されていません")

    def run_processing(self):
        if 'value' not in self.file_lists or not self.file_lists['value']:
            messagebox.showerror("エラー", "基準となるグラフ1(value)のファイルを最低1つは選択してください。")
            return
        
        # ★ 'value2' が有効 (enabled) なのにファイルが選択されていない場合のエラーチェック
        if 'value2' in self.graph_settings_vars and self.graph_settings_vars['value2']['enabled'].get():
             if 'value2' not in self.file_lists or not self.file_lists['value2']:
                 messagebox.showerror("エラー", "グラフ 2(value2) が有効ですが、ファイルが選択されていません。")
                 return
        
        # ★ 他の有効なグラフについてもファイル存在チェック (オプション)
        for key, vars in self.graph_settings_vars.items():
            if key != 'value' and vars['enabled'].get():
                if key not in self.file_lists or not self.file_lists[key]:
                    messagebox.showerror("エラー", f"グラフ {key} が有効ですが、ファイルが選択されていません。")
                    return

        try:
            params = {key: var.get() for key, var in self.params.items()}
            
            graph_configs = {}
            for key, vars in self.graph_settings_vars.items():
                if vars['enabled'].get():
                    # ウィジェットオブジェクトや不要なキーを除外して設定ディクショナリを作成
                    config_data = {}
                    for v_key, v_var in vars.items():
                        if not isinstance(v_var, (tk.Widget, ttk.Widget)):
                             config_data[v_key] = v_var.get()
                    graph_configs[key] = config_data

            processor = DataProcessor(self.file_lists, params, graph_configs)
            processor.process_all_files()
            messagebox.showinfo("完了", "処理が完了しました。")
        except Exception as e:
            messagebox.showerror("エラー", f"処理中にエラーが発生しました:\\n{e}")
            import traceback
            traceback.print_exc()

class DataProcessor:
    def __init__(self, file_lists, params, graph_configs):
        self.file_lists = file_lists
        self.params = params
        self.graph_configs = graph_configs # GUIから渡された設定 (enabled なものだけ)
        self.jst = pytz.timezone('Asia/Tokyo')

    def process_all_files(self):
        for key in self.file_lists:
            self.file_lists[key] = self.natural_sort(self.file_lists[key])
            
        # value ファイルの数に基づいてループ
        num_files_value = len(self.file_lists.get('value', []))
        
        for i in range(num_files_value):
            current_files = {'value': self.file_lists['value'][i]}
            
            # 他の有効なグラフについて、対応するインデックスのファイルを取得
            for key in self.graph_configs:
                if key != 'value':
                    if key in self.file_lists and i < len(self.file_lists[key]):
                        current_files[key] = self.file_lists[key][i]
                    else:
                        # ファイルが存在しない (数が合わないか、選択されていない)
                        print(f"Warning: File for enabled graph '{key}' not found at index {i}. Skipping this graph for this set.")
                        
            self.process_single_set(current_files)

    def natural_sort(self, l):
        convert = lambda text: int(text) if text.isdigit() else text.lower()
        alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
        return sorted(l, key=alphanum_key)

    def process_single_set(self, file_paths):
        all_data = {}
        # file_paths には、このインデックス (i) でファイルが存在するグラフのパスのみが入っている
        for key, path in file_paths.items():
            if key in self.graph_configs: # enabled なグラフの設定のみ
                config = self.graph_configs[key]
                all_data[key] = self.data_format(path, config['x_axis'], config['y_axis'])
        
        if 'value' not in all_data or not all_data['value']['timestamp'].any():
            print(f"Skipping set for {file_paths.get('value', 'N/A')} - No 'value' data found or data is empty.")
            return

        value_data = all_data['value']
        csv_name = value_data['csv_name']
        timestamp = value_data['timestamp']

        # Create a main 'graph' directory if it doesn't exist
        base_output_dir = os.path.join(os.getcwd(), "graph")
        if not os.path.exists(base_output_dir):
            os.makedirs(base_output_dir)

        output_folder_name = f"graph_{self.params['ver_number']}_{csv_name}"
        output_path = os.path.join(base_output_dir, output_folder_name)
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        if len(timestamp) == 0:
             print(f"Skipping set for {csv_name} - 'value' timestamp is empty.")
             return
             
        timestamp_f = timestamp[0]
        timesp_start, timesp_end = self.separate_timestamp_startend(timestamp_f, timestamp)
        
        for x in range(len(timesp_start)):
            valid_data_dict = {}
            # 'value' グラフのデータを取得
            valid_timestamp, valid_value = self.sp_data_to_graph(x, timesp_start, timesp_end, timestamp, value_data['value'])
            
            if not len(valid_timestamp):
                continue
                
            valid_data_dict['value'] = {'timestamp': valid_timestamp, 'value': valid_value}

            # all_data には file_paths に存在した enabled なグラフのデータが入っている
            for key, data in all_data.items():
                if key != 'value':
                    ts, val = self.sp_data_to_graph(x, timesp_start, timesp_end, data['timestamp'], data['value'])
                    if len(ts) > 0: # この区間にデータがある場合のみ追加
                        valid_data_dict[key] = {'timestamp': ts, 'value': val}
            
            # valid_data_dict には、この区間 (x) でデータが存在するグラフのみが入る
            
            # self.graph_configs (enabled な全グラフ設定) を渡す
            self.graph_create(x, output_path, csv_name, valid_data_dict, self.graph_configs)
            self.combine_csv(x, output_path, csv_name, valid_data_dict, self.graph_configs)

    def data_format(self, file_path, x_col, y_col):
        csv_name = os.path.splitext(os.path.basename(file_path))[0]
        try:
            df_data = pd.read_csv(file_path, encoding='utf-8')
            if df_data.empty:
                return {'csv_name': csv_name, 'timestamp': np.array([]), 'value': np.array([])}
            timestamp = df_data.iloc[:, x_col].to_numpy()
            value = df_data.iloc[:, y_col].to_numpy()
            return {'csv_name': csv_name, 'timestamp': timestamp, 'value': value}
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return {'csv_name': csv_name, 'timestamp': np.array([]), 'value': np.array([])}


    def separate_timestamp_startend(self, timestamp_f, timestamp):
        """
        タイムスタンプを指定した時間で区切ります。
        最後の区切りが指定されたdurationよりも短くても、
        全体の終了時刻までデータを含めるように調整されます。
        """
        shift_time = self.params['shift_time']
        duration = self.params['duration']

        if len(timestamp) == 0:
            return np.array([]), np.array([])
            
        last_overall_time = float(timestamp[-1])
        
        # データ全体の長さがdurationより短い場合は、全体を1つの区間とする
        if last_overall_time - timestamp_f <= duration:
            return np.array([timestamp_f]), np.array([last_overall_time])

        timestamp_sp_start = []
        current_time = timestamp_f
        
        while True:
            end_time = current_time + duration
            
            # 終了時間が全体の終了時間を超える場合
            if end_time >= last_overall_time:
                # 最後の区間を追加して終了
                timestamp_sp_start.append(max(timestamp_f, last_overall_time - duration))
                break
            
            timestamp_sp_start.append(current_time)
            current_time += shift_time

        timestamp_sp_end = [start + duration for start in timestamp_sp_start[:-1]]
        timestamp_sp_end.append(last_overall_time) # 最後の区間の終了はデータの最後

        # 重複を削除して返す
        unique_starts, unique_indices = np.unique(timestamp_sp_start, return_index=True)
        unique_ends = np.array(timestamp_sp_end)[unique_indices]

        return unique_starts, unique_ends

    def sp_data_to_graph(self, x, time_sp_start, time_sp_end, timestamp, value):
        time_indices = np.where((timestamp >= time_sp_start[x]) & (timestamp <= time_sp_end[x]))[0]
        return timestamp[time_indices], value[time_indices]

    def from_timestamp_to_datetime(self, timestamp):
        utc_time = dt.utcfromtimestamp(timestamp / 1000)
        jst_time = pytz.utc.localize(utc_time).astimezone(self.jst)
        return jst_time.strftime('%Y-%m-%d %H:%M:%S') + f".{int(timestamp % 1000):03d}"

    def combine_csv(self, x, output_path, csv_name, valid_data_dict, graph_configs):
        # ★ CSV出力形式を「横並び＋空列スペーサー」に変更
        
        value_data = valid_data_dict.get('value', {})
        if not value_data or value_data.get('timestamp') is None or len(value_data['timestamp']) == 0:
             return
        
        df_list = []
        max_len = 0 # この区間で最もデータ点数が多いグラフの行数を保持
        
        # 1. データを集め、最大行数を探す
        # graph_configs (enabled な全グラフ) の順序 (GUIのタブ順) で DataFrame を作成
        for key in graph_configs.keys():
            if key in valid_data_dict: # この区間にデータがあるか
                data = valid_data_dict[key]
                # data が None や空でないことを再確認
                if data and data.get('timestamp') is not None and len(data['timestamp']) > 0:
                     df = pd.DataFrame({
                        # 列名を "timestamp" と グラフ名 にする
                        "timestamp": data['timestamp'],
                        graph_configs[key]['name']: data['value']
                    })
                     df_list.append(df)
                     if len(df) > max_len:
                        max_len = len(df)
        
        if not df_list:
            return # 有効なデータがなかった

        # 2. 空のDataFrame (スペーサー) を定義
        # 最大行数 (max_len) に合わせた長さで作成
        # 列名を 1文字の空白 ' ' にすると、CSV出力時に空列 (ヘッダも空) に見える
        spacer_df = pd.DataFrame(np.nan, index=range(max_len), columns=[' '])

        # 3. 連結リストを作成
        final_df_list_to_concat = []
        for df in df_list:
            # 各DFの行数が max_len に満たない場合、足りない行を NaN で埋める (reindex)
            df_reindexed = df.reindex(range(max_len))
            
            final_df_list_to_concat.append(df_reindexed)
            final_df_list_to_concat.append(spacer_df) # スペーサーを追加
            
        # 最後のスペーサーを削除
        if final_df_list_to_concat:
            final_df_list_to_concat.pop()
            
        # 4. 連結 (axis=1 で横方向に連結)
        combined_df = pd.concat(final_df_list_to_concat, axis=1)
        
        # --- ファイル名作成 ---
        f_time_dt = self.from_timestamp_to_datetime(value_data['timestamp'][0])
        last_time_dt = self.from_timestamp_to_datetime(value_data['timestamp'][-1])
        f_time_str = f_time_dt.split(' ')[1].replace(":", "-")
        last_time_str = last_time_dt.split(' ')[1].replace(":", "-")

        file_name = f"{x}_{f_time_str}_{last_time_str}_{csv_name}.csv"
        file_path = os.path.join(output_path, file_name)
        
        # index=False でインデックス (0, 1, 2...) をCSVに出力しない
        combined_df.to_csv(file_path, index=False)

    def graph_create(self, x, output_path, csv_name, valid_data_dict, graph_configs):
        # ★ 描画ロジックを「第2軸使用」チェックボックス対応に変更
        
        output_graph_path = os.path.join(output_path, "graph")
        if not os.path.exists(output_graph_path):
            os.makedirs(output_graph_path)

        fig, ax1 = plt.subplots(figsize=(15, 6))
        
        lines = []
        secondary_axes = [] # ax1.twinx() で作成した軸を保持
        
        # graph_configs (enabled な全グラフ) の順序でプロット
        # (GUIのタブ順 (value, value2, peak, ...) になる)
        plot_order = list(graph_configs.keys())

        primary_ylim = None # 主軸 (ax1) の Y レンジ (固定の場合)
        
        for key in plot_order:
            # この区間にデータがあるグラフのみ描画
            if key not in valid_data_dict:
                continue
                
            data = valid_data_dict[key]
            config = graph_configs[key] # enabled なグラフの設定
            
            # data が None や空でないことを再確認
            if not (data and data.get('timestamp') is not None and len(data['timestamp']) > 0):
                continue
                
            datetimes = [pd.to_datetime(self.from_timestamp_to_datetime(ts)) for ts in data['timestamp']]
            
            # 'use_secondary_y' は graph_configs に GUI から渡されている
            use_secondary = config.get('use_secondary_y', False)
            
            current_ax = None
            if not use_secondary: # 主軸 (ax1) を使用
                current_ax = ax1
                if not config['autorange'] and primary_ylim is None:
                    # このグラフが主軸を使い、固定レンジで、まだ主軸レンジが設定されていない
                    # (plot_order の先頭 (通常 'value') の設定が優先される)
                    primary_ylim = (config['yrange_min'], config['yrange_max'])
            
            else: # 第2軸 (以降) を使用
                current_ax = ax1.twinx()
                secondary_axes.append(current_ax)
                
                if len(secondary_axes) > 1: # 2つ目以降の twinx (つまり第3軸以降)
                    # 軸を右側にオフセット
                    current_ax.spines['right'].set_position(('outward', 60 * (len(secondary_axes) - 1)))
                
                # 第2軸 (以降) の Y レンジ設定
                if not config['autorange']:
                    current_ax.set_ylim(config['yrange_min'], config['yrange_max'])
                
                # ★ Y軸ラベルの色を 'black' に変更
                current_ax.set_ylabel(config['name'], color='black')
                # ★ Y軸の目盛りと軸の色を 'black' に変更し、目盛りを内向き ('in') に設定
                current_ax.tick_params(axis='y', labelcolor='black', colors='black', direction='in')
                current_ax.spines['right'].set_edgecolor('black')


            # Plot style based on GUI selection
            style = config.get('plot_style', 'Line')
            plot_args = {'label': config['name'], 'color': config['color']}
            
            if style == 'Line':
                line, = current_ax.plot(datetimes, data['value'], **plot_args)
            elif style == 'Marker':
                plot_args.update({'linestyle': 'none', 'marker': 'o'})
                line, = current_ax.plot(datetimes, data['value'], **plot_args)
            elif style == 'Line + Marker':
                plot_args.update({'marker': 'o'})
                line, = current_ax.plot(datetimes, data['value'], **plot_args)
            
            lines.append(line)

        # --- ループ終了 ---

        # value データがこの区間に存在しない場合は、グラフを作成しない
        value_data = valid_data_dict.get('value', {})
        if not value_data or value_data.get('timestamp') is None or len(value_data['timestamp']) == 0:
            plt.close(fig)
            return

        # 主軸 (ax1) の Y レンジを適用
        if primary_ylim:
            ax1.set_ylim(primary_ylim[0], primary_ylim[1])

        # 主軸 (ax1) の Y ラベルを設定
        # 主軸を使ったグラフの名前をリストアップ
        primary_labels = [graph_configs[k]['name'] for k in plot_order if k in valid_data_dict and not graph_configs[k].get('use_secondary_y', False)]
        primary_colors = [graph_configs[k]['color'] for k in plot_order if k in valid_data_dict and not graph_configs[k].get('use_secondary_y', False)]
        
        if primary_labels:
            # ★ Y軸ラベルの色を 'black' に変更
            ax1.set_ylabel(" / ".join(primary_labels), color='black')
            # 主軸の Y ラベルとティックの色を、主軸の *最初の* グラフの色に合わせる (通常 'value')
            if primary_colors:
                # ★ Y軸の目盛りと軸の色を 'black' に変更し、目盛りを内向き ('in') に設定
                ax1.tick_params(axis='y', labelcolor='black', colors='black', direction='in')
                ax1.spines['left'].set_edgecolor('black')
                ax1.yaxis.label.set_color('black')
        else:
             # 主軸にグラフがない場合 (value が無効化されることはないが、念のため)
             # ★ 目盛りを内向き ('in') に設定
             ax1.tick_params(axis='y', direction='in')


        
        if lines:
            # 凡例をプロットの下中央に配置
            ax1.legend(lines, [l.get_label() for l in lines], loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(lines))
        
        ax1.set_xlabel('datetime')
        
        # Set major and minor grids
        ax1.grid(which='major', linestyle='--', linewidth='0.7', color='gray')
        
        x_minor_tick_interval = self.params.get('x_minor_tick_interval', 0)
        if x_minor_tick_interval > 0:
            ax1.xaxis.set_minor_locator(mdates.SecondLocator(interval=x_minor_tick_interval))
            ax1.grid(which='minor', linestyle=':', linewidth='0.5', color='lightgray')


        f_time_dt = self.from_timestamp_to_datetime(value_data['timestamp'][0])
        last_time_dt = self.from_timestamp_to_datetime(value_data['timestamp'][-1])
        f_time_str = f_time_dt.split(' ')[1].replace(":", "-")
        last_time_str = last_time_dt.split(' ')[1].replace(":", "-")
        
        plt.title(f"{x}_{f_time_str}_{last_time_str}_{csv_name}")
        
        output_file = os.path.join(output_graph_path, f"{x}_{f_time_str}_{last_time_str}_{csv_name}.jpg")
        # bbox_inches='tight' で凡例や軸ラベルが画像外に切れるのを防ぐ
        plt.savefig(output_file, format='jpg', bbox_inches='tight')
        plt.close(fig)

if __name__ == "__main__":
    app = GraphCreatorApp()
    app.mainloop()